In [1]:
import numpy as np
import tensorflow as tf

In [2]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

b[:, -1]

array([ 4,  8, 12])

In [3]:
b[-1]

array([ 9, 10, 11, 12])

In [4]:
b[-1, :]

array([ 9, 10, 11, 12])

In [5]:
b[-1, ...]

array([ 9, 10, 11, 12])

In [6]:
b[0:2, :]

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [7]:
b[:, 0:2]

array([[ 1,  2],
       [ 5,  6],
       [ 9, 10]])

In [16]:
xy = np.loadtxt("data-01-test-score.csv", delimiter=",", dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())
# Set up feed_dict variables inside the loop.
for step in range(2001):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train],
        feed_dict={X: x_data, Y: y_data})
    
    if step % 1000 == 0:
        print(step, f"Cost: {cost_val}\nPrediction: \n{hy_val}\n\n")
        
# Ask my score
print("Your score will be ", sess.run(hypothesis, 
                                      feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ", sess.run(hypothesis,
                                        feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

0 Cost: 9.098071098327637
Prediction: 
[[151.87302]
 [183.25195]
 [180.03029]
 [196.76895]
 [139.58992]
 [104.54149]
 [148.48811]
 [111.62869]
 [172.52612]
 [161.73126]
 [142.55786]
 [141.29878]
 [185.15714]
 [153.02148]
 [149.39359]
 [186.61734]
 [145.10347]
 [178.27164]
 [176.30054]
 [157.90536]
 [173.78804]
 [172.70291]
 [165.43668]
 [151.24643]
 [189.35617]]


1000 Cost: 6.740566253662109
Prediction: 
[[153.05548 ]
 [184.78238 ]
 [181.46942 ]
 [198.49107 ]
 [140.65616 ]
 [105.57846 ]
 [149.97623 ]
 [113.007095]
 [174.1705  ]
 [163.6058  ]
 [143.86513 ]
 [142.69508 ]
 [186.4873  ]
 [153.91275 ]
 [150.92194 ]
 [188.30782 ]
 [145.79807 ]
 [180.13042 ]
 [177.52722 ]
 [159.01907 ]
 [175.49399 ]
 [174.28268 ]
 [167.00972 ]
 [152.07443 ]
 [190.8522  ]]


2000 Cost: 6.533350944519043
Prediction: 
[[152.9915  ]
 [184.738   ]
 [181.4005  ]
 [198.52094 ]
 [140.54341 ]
 [105.660286]
 [150.14496 ]
 [113.33604 ]
 [174.26439 ]
 [163.92714 ]
 [143.92958 ]
 [142.81297 ]
 [186.3092  ]
 [153.61641 ]


In [21]:
# 파일이 너무 커서 메모리에 올리기 어려운 경우, 텐서플로우에서는 Queue Runner 시스템을 만들어 두었음

filename_queue = tf.train.string_input_producer(
    ["data-01-test-score.csv"],
    shuffle=False,
    name="filename_queue"
)

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [[0], [0], [0], [0]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

sess = tf.Session()

# Start populating the filename queue
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    
coord.request_stop()
coord.join(threads)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
